The model mountain...
in silico, it is a 2D array
row index - altitudinal band - how far up the mountain
column index - how far along an altitudinal band

# Dispersal
(  
The model mountain is a circle sector and the polar coordinates, r and theta, describe a position.
- The radial coordinate, r, is the radial distance from the circle centre (mountain tip) - how far up the mountain a point is - its altitudinal band or row index.
- The angular coordinate, theta, is the angle from the x-axis - how far along an altitudinal band a point is - its column index.

*issue*  
)

An individual dies, leaving a gap, which is filled, via dispersal, by the offspring of another individual.
The dispersal:
- destination is the vacant cell (vacant due to death)
- start point is the parent's cell.

The program:
- randomly picks the parent from a probability distribution - the probability of each parent's offspring dispersing to the vacant position
- calculates probabilities using the individuals' dispersal kernels.

Individuals do not have the same chance of being chosen, as their dispersal ability varies (i.e. the probability distribution is not uniform). (The shape of dispersal kernels varies. It depends on the individual's size and the temperature of its position.)

Dispersal occurs by randomly picking a distance along each axis.
To apply a metabolic effect to dispersal ability, calculate distance as follows:
- Randomly pick a number from a normal distribution (location 0, scale 1).
- Multiply the number by a variable, 'x', which is the output of metabolic scaling.
- Distance is a continuous variable, but the landscape is divided into discrete units (position is a discrete variable). So, use the floor of the distance (round it down to the next integer)$^1$.

However, the program does this in reverse, as it must calculate dispersal probabilities before it runs the simulation.

$^1$ I do not round up as, if I did, only a raw distance of 0 would give a rounded distance of 0 (any raw distance greater than 0 and less than or equal 1, would round to 1).

To go from one point to another, offspring obviously must move the distance between the points. However, this is the minimum distance they must go; they may go a more tortuous route, theoretically up to an infinite distance. So, the probability of dispersing from one point to another, is the probability of going *at least* the distance between them. This is calculated as:  
&nbsp;
$$P(d\geq d_{min}) = 1 - P(d < d_{min})$$
&nbsp;  
where $d$ is the distance an offspring moves and $d_{min}$ is the distance between the points.

(The program uses the floor of the distance. So, a distance, n, in number of cells represents a range of distances: $n\leq d < n + 1$, where $d$ is raw distance. The program accounts for this, as it calculates the probability of going at least the minimum distance.)

## Dispersal Map
Each cell in the simulated landscape has a dispersal map, containing the probability of dispersing to that cell, from every cell (i.e. a probability distribution). The algorithm that computes a dispersal map:
- Separately considers the destination's r and theta position (i.e., it separately considers dispersal along each axis).
- Makes an array containing, for each r point in the landscape, the probability of dispersing to the destination r.
- Does a similar computation for theta.
- Multiplies the arrays element-wise, to get, for each (r, theta) position, the probability of dispersing to the destination.

## Area
- A mountain base covers more area than the top.
- While the model landscape is a circle sector, in silico, it is a rectangular array. Each row is an altitudinal band.
- So, going up the mountain, each grid cell represents an ever narrower area.
- Thus, the theta distance must be scaled according to the radial position.
- Movement in theta follows an arc of a circumference, so divide distance by circumference length, $2\pi r$ (the radial position gives the radius).
- The program separately considers dispersal along each axis. Whether it considers r or theta first matters, as the theta distance is scaled according to the radial position. *I assume it is sufficient to calculate theta distance, using the radial position halfway between the start and destination - the average.*

*diagrams*

In [6]:
# code

Do the following for each axis:
- Make an array of all indices along the axis (position of each potential parent).
- For each index:
    - Calculate the distance to the destination (minimum distance offspring must go).
    - Divide by x (then, for theta, multiply by $2\pi r$) to get the variate of the normal distribution that gives this distance.
    - The probability of going at least this distance is $1 - P(d < d_{min})$, where $d$ is distance an offspring moves and $d_{min}$ is distance between the points.  
    `stats.halfnorm.sf(x)` - survival function (1 - `cdf`) evaluated at x.

I do not assume offspring go in a fixed direction (see 'Change to Implementation'). So, I use the distance's absolute value, and thus a half normal distribution.

- Dispersal in theta is along axis 1 (across columns).
- For theta, distance must be scaled according to the radial position.
- So, repeat the calculation for each row (altitudinal band). (Thus, for axis 1, the array of probabilities is 2D.)
- `radii` contains the radial position of each row. (Radial distance in number of cells from the circle centre/mountain tip to the centre of an altitudinal band).

## Change to Implementation
- *As the landscape's left and right edges are connected, dispersal in theta can occur via any number of loops around the mountain. As discussed, I should not assume offspring go the shortest way, when calculating the probability of going from one point to another.*
- *So they do not over- or undershoot the destination, offspring must do a full loop (though they may do any number of full loops). Thus, unlike my current implementation, offspring cannot go infinite distances greater than the minimum. I wrote code for this, but:*
- *I feel I should also not assume offspring go in a fixed direction. (Note I am considering dispersal along one axis, so there are only two directions). Nothing stops offspring going, e.g., forward 2 steps, back 1, forward 3. As long as they go back on themselves or do a full loop, they reach the destination.*
- *Given also dispersal is a continuous variable, we now get truly infinite possible distances above the minimum.*